#### 自然言語処理の実行手順
1. クリーニング - 文章のデータではないノイズの除去
1. 正規化 - 全角半角の統一など
1. 単語分割 - 文章を単語ごとに区切る
1. 基本形への変換 - 「走っ」を「走る」　など
1. ストップワード除去 - 助詞など意味を持たない文字を消去
1. 単語の数値化　- IDをふったりベクトルに変換する

In [14]:
import torch
import torch.nn as nn
import torchtext
import torchdata.datapipes as dp
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import torchtext.transforms as T
import re
from janome.tokenizer import Tokenizer
import pandas as pd

In [15]:
class Config():
    batch_size=128
    n_epoch=20
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    emb_dim = 100
    hid_dim = 50
    train_path='./data/text_train.tsv'
    valid_path='./data/text_valid.tsv'
    test_path='./data/text_test.tsv'
config=Config()

print(config.device)

cuda:0


In [16]:
df_train=pd.read_csv(config.train_path,delimiter='\t',names=['text','Label'])
df_valid=pd.read_csv(config.valid_path,delimiter='\t',names=['text','Label'])
df_test=pd.read_csv(config.test_path,delimiter='\t',names=['text','Label'])


In [48]:
class MakeDataLoader():
    def __init__(self,df,batch_size=4,train=True):
        self.df=df
        self.batch_size=batch_size
        self.train=train
    #tokenize_preprosessでコレを指すのでselfが引数に必要
    def tokenizer_janome(self,text):
        j_t=Tokenizer()
        return [tok for tok in j_t.tokenize(text,wakati=True)]
    
    def preprocessing_text(self,text):
        text=re.sub('\r', '', text)#改行を消去
        text=re.sub('\n', '', text)
        text=re.sub(' ', '', text)#半角スペースを消去
        text=re.sub('　', '', text)#全角スペースを消去
        text=re.sub(r'[0-9 ０-９]', '0' , text)#数字を一律0に変換

        return text
    
    def tokenize_preprosess(self,text):
        text=self.preprocessing_text(text)
        ret=self.tokenizer_janome(text)
        return ret
    
    def run_tokenize(self):
        self.df['text']=self.df['text'].map(lambda text:' '.join(text))
        self.df['text']=self.df['text'].map(lambda text : self.tokenize_preprosess(text))

    def make_vocab(self):
        self.text_vocab=build_vocab_from_iterator(self.df['text'],specials=('<unk>','<pad>'))
        self.text_vocab.set_default_index(self.text_vocab['<unk>'])
    
    def label_assign(self):
        self.df['Label'] = self.df['Label'].astype(str)  # ラベルが数値のため文字型に変換。文字型で指定していた場合はこの処理は不要
        self.label_vocab = build_vocab_from_iterator(self.df['Label'])

    def make_transform(self):
        text_transform=T.Sequential(
            T.VocabTransform(self.text_vocab),
            T.ToTensor(padding_value=self.text_vocab['<pad>'])
            )
        label_transform=T.Sequential(
            T.VocabTransform(self.label_vocab),
            T.ToTensor()
        )
        return text_transform,label_transform

    def collate_batch(self,batch):
        texts,_=self.make_transform([text for (text,label) in batch])
        _,labels=self.make_transform([label for (text, label) in batch])
        return texts,labels

    def dataload(self):
        return DataLoader(self.df.values,
                        batch_size=self.batch_size,
                        collate_fn=self.collate_batch)
dataloader=MakeDataLoader(df=df_train)
dataloader_train=dataloader.dataload()

#dataloader の昨日の確認とword2vecの実装までする


In [49]:
for i ,(texts,labels) in enumerate(dataloader_train):
    print(i)
    for text,label in zip(texts,labels):
        print(text,label_vocab.lookup_token(label))

TypeError: MakeDataLoader.make_transform() takes 1 positional argument but 2 were given

#### 単語の数字への変換
1. tokenに対してIDを割り振って学習させる方法<br>
→情報が離散的で近くにある文字しか対象にならない
1. word2vecなどを用いてベクトルに変換する<br>
→ベクトルにしているので遠くの単語の類似度もわかる

In [18]:
# #tokenからword2vecを用いてベクトルに変換する
# #東北大学の乾研究室の学習済みデータを用いる
# #https://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/

# #! conda install anaconda::gensim
# import os
# import urllib.request
# import zipfile
# import tarfile

# from gensim.models import KeyedVectors

# url = "http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"
# save_path = "./data/20170201.tar.bz2"
# if not os.path.exists(save_path):
#     urllib.request.urlretrieve(url, save_path)

# # tarファイルを読み込み
# tar = tarfile.open('./data/20170201.tar.bz2', 'r|bz2')
# tar.extractall('./data/')  # 解凍
# tar.close()  # ファイルをクローズ

# model = KeyedVectors.load_word2vec_format(
#     './data/entity_vector/entity_vector.model.bin', binary=True)

# # 保存（時間がかかります、10分弱）
# model.save_word2vec_format('./data/japanese_word2vec_vectors.vec')

In [19]:
from torchtext.vocab import Vectors

japanese_word2vec_vectors=Vectors(
    name='./data/japanese_word2vec_vectors.vec'
)

print('１単語の次元 : ',japanese_word2vec_vectors.dim)
print('単語数 : ',len(japanese_word2vec_vectors.itos))

１単語の次元 :  200
単語数 :  1015474


In [40]:
results = model.most_similar(u'[前田敦子]')
for result in results:
    print(result)

('[高橋みなみ]', 0.9047951698303223)
('[大島優子]', 0.8955446481704712)
('[松井珠理奈]', 0.8776582479476929)
('[松井玲奈]', 0.8701295256614685)
('[指原莉乃]', 0.8524861335754395)
('[島崎遥香]', 0.8492460250854492)
('[山本彩]', 0.8412748575210571)
('[小嶋陽菜]', 0.835233747959137)
('[渡辺麻友]', 0.8340295553207397)
('[篠田麻里子]', 0.8066893815994263)


In [21]:
results = model.most_similar(positive=[u'[妻]'],negative=[u'[愛人]'])
for result in results:
    print(result)


('ドレナージ', 0.4027651846408844)
('[分娩]', 0.39615094661712646)
('[座位]', 0.3796175420284271)
('[出産]', 0.37935060262680054)
('[生殖]', 0.3761950731277466)
('望ましい', 0.37324923276901245)
('[子育て]', 0.372659832239151)
('[歯科医師]', 0.3716592788696289)
('[歯学者]', 0.36931660771369934)
('[公認心理師]', 0.36690229177474976)
